In [14]:
import pandas as pd
import numpy as np
import os
from dateutil import parser
from collections import OrderedDict
from cassandra.cluster import Cluster
import logging 
from time import time
import datetime
from dateutil import parser

data_dir = 'X:\\Data_Analytics\\BankNifty\\data\\'
download_dir = 'X:\\Data_Analytics\\BankNifty\\download\\'
master_dir = 'X:\\Data_Analytics\\BankNifty\\Master\\'

In [11]:




result = pd.DataFrame()
week_df = pd.DataFrame()
timestamp_df = []
for r,d,f in os.walk(download_dir):
    
    for csv_file in f:
        print csv_file
        dates = OrderedDict()
        df = pd.read_csv(data_dir+csv_file)
        df = df[df['SYMBOL']=='BANKNIFTY']
        monthly = df[df['INSTRUMENT']=='FUTIDX']
        timestamp = parser.parse(monthly['TIMESTAMP'][0])
        #monthly['EXPIRY_DT'] = monthly.apply(lambda row: pd.datetime.date(parser.parse(row['EXPIRY_DT'])), axis=1)        
        monthly = list(set(monthly['EXPIRY_DT']))
        result = result.append(monthly)
        timestamp_df.append(timestamp)
      
        
        
        
        weekly = df[df['INSTRUMENT']=='OPTIDX']
        
        #weekly['EXPIRY_DT'] = weekly.apply(lambda row: pd.datetime.date(parser.parse(row['EXPIRY_DT'])), axis=1)
        weekly = list(set(weekly['EXPIRY_DT']))
        week_df = week_df.append(weekly)
        
    
result.to_excel('month.xlsx')
week_df.to_excel('weeks.xlsx')  
timestamp_df = pd.DataFrame(timestamp_df)
timestamp_df.to_excel('timestamp.xlsx')               
        
        
        
        
        
        
        

fo01AUG2018bhav.csv
fo26NOV2018bhav.csv


In [10]:
result.to_excel('dates.xlsx')

In [16]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)


def get_values(min_date, max_date, key):
    '''Function to retrive values from cassandra db'''
    
    cluster = Cluster(['127.0.0.1'])
    logging.info('Cassandra Cluster connected...')
    # connect to your keyspace and create a session using which u can execute cql commands 
    session = cluster.connect('test_df')
    logging.info('Using test_df keyspace')
    #define rows fetched from cassandra to be a pandas dataframe
    session.row_factory = pandas_factory
    session.default_fetch_size = None    

    query = 'SELECT * FROM test_bhavcopy WHERE price_date >= \'{0}\' AND price_date <= \'{1}\' AND key = \'{2}\'ALLOW FILTERING;'.format(
                        min_date,max_date,key)

    rows = session.execute(query, timeout = None)
    rows = rows._current_rows
    rows.drop(columns=['key','option_typ','strike_pr'], inplace=True)
    return rows
    
def anchor_buckets_func(value, anchor_date, days_index, symbol, df):
    '''Function to get bucket values'''
    
    dates = df['price_date']
    result = OrderedDict()
    # before anchor date
    anchor_value = df[df['price_date'] == anchor_date][value].values[0]
    
    for index in list(reversed(days_index)):
        try:
            result['X-{0}({1})'.format(index, value)] = df[df['price_date'] == dates[dates[dates == anchor_date].index[0] - index].date()][value].values[0] 
        except:
            logging.info('Not in index {0}'.format(index))
    

    result['X({0})'.format(value)] = anchor_value

    # after anchor date
    for index in days_index:
        try:
            result['X+{0}({1})'.format(index, value)] = df[df['price_date'] == dates[dates[dates == anchor_date].index[0] + index].date()][value].values[0] 
        except:
            logging.info('Not in index {0}'.format(index))

    res = pd.DataFrame(result, index=[anchor_date], columns=result.keys())
    
    return res


def min_max_calc(min_date, max_date, anchor_date):
    '''Func to handle absolute days for min and max dates'''
    min_date = min_date
    max_date = max_date
    if len(min_date)<=3:
        # handle abs days
        min_date = anchor_date - datetime.timedelta(days=int(min_date))
    else:
        try:
            min_date = parser.parse(min_date).date()
        except:
            logging.info('Out of range for current month')
        
    if len(max_date)<=3:
        #handle abs days
        max_date = anchor_date + datetime.timedelta(days=int(max_date))
    else:
        try:
            max_date = parser.parse(max_date).date()
        except:
            logging.info('Out of range for current month')
        
    return min_date, max_date
        
        

In [3]:
# Get inputs
quarter_dates = [parser.parse('3/31/2018'), parser.parse('6/30/2018'),parser.parse('9/30/2018'), parser.parse('12/30/2018')]

value, anchor_date,symbol = str.lower('Close'),parser.parse('22 Nov 2018').date(),str.upper('ACC')

days_index = [2,4,6,7]

#min_date = raw_input('min date')
#max_date = raw_input('max date')

#min_date,max_date = min_max_calc(min_date, max_date, anchor_date)

# fetch keys from master 
keys = pd.read_excel(master_dir+'MasterData.xlsx')
keys = keys[keys['IsActiveFNO']==True]
keys.loc[keys['Type']=='SSF','instrument'] = 'FUTSTK'
keys.loc[keys['Type']=='Index','instrument'] = 'FUTIDX'
keys = keys['SYMBOL']+"_"+keys['instrument']+"_0.0_XX_1"

# fetch key for symbol
key = keys[keys.str.startswith(symbol+'_')].values[0]

# get values from cassandra
df = get_values(min_date,max_date,key)
df = df[['symbol','price_date',value]]

# call buckets func
result = anchor_buckets_func(value, anchor_date, days_index, symbol, df)


min date7
max date7


X-4     X-2       X     X+2      X+4
ACC  1507.55  1475.0  1461.8  1446.9  1462.45

In [5]:
result

X-4    X-2       X   X+2    X+4
ACC  -45.75  -13.2  1461.8  14.9  -0.65

In [321]:
df

symbol  price_date open_int
0    ACC  2018-11-15   885600
1    ACC  2018-11-16   895600
2    ACC  2018-11-19  1057200
3    ACC  2018-11-20  1055200
4    ACC  2018-11-21  1126000
5    ACC  2018-11-22  1124800
6    ACC  2018-11-26  1225200
7    ACC  2018-11-27  1009200
8    ACC  2018-11-28   754000
9    ACC  2018-11-29   397200

In [85]:
symbol = str.upper('BANKNIFTY')
anchor_dates = pd.read_excel('input.xlsx')
value = 'close'
min_date, max_date = min(anchor_dates['Dates']).date() , max(anchor_dates['Dates']).date()

anchor_dates['Dates'] = anchor_dates.apply(lambda row: row['Dates'].date(), axis=1)


#min_date = raw_input('min date')
#max_date = raw_input('max date')
#min_date,max_date = min_max_calc(min_date, max_date, anchor_date)

# fetch keys from master 
keys = pd.read_excel(master_dir+'MasterData.xlsx')
keys = keys[keys['IsActiveFNO']==True]
keys.loc[keys['Type']=='SSF','instrument'] = 'FUTSTK'
keys.loc[keys['Type']=='Index','instrument'] = 'FUTIDX'
keys = keys['SYMBOL']+"_"+keys['instrument']+"_0.0_XX_1"

# fetch key for symbol
key = keys[keys.str.startswith(symbol+'_')].values[0]


# get values from cassandra
df = get_values(min_date,max_date,key)
#df = df[['symbol','price_date',value]]
df.to_excel('df.xlsx')
days_index = [1]
# get anchor+-1 close prices 
close_df = pd.DataFrame()
for anchor_date in anchor_dates['Dates']:
    close_df = close_df.append(anchor_buckets_func('close', anchor_date, days_index, symbol, df))
    
close_df.index.name = symbol

# get anchor+-1 close prices
open_df = pd.DataFrame()
for anchor_date in anchor_dates['Dates']:
    open_df = open_df.append(anchor_buckets_func('open', anchor_date, days_index, symbol, df))
    
open_df.index.name = symbol
# get only X+1 open values 
open_df = open_df[['X+1(open)']]
# merge both the df
final_df = close_df.merge(open_df, on=symbol)
final_df.fillna(0, inplace=True)
final_df.reset_index(inplace=True)



# get report 
report_df = pd.DataFrame(columns=['A_close to B-1_close','B-1_close to B_close', 'A+1_open to B-1_close' ])
anchor_dates = final_df.index

i = 0
for anchor_date in final_df[symbol]:
    res = {}
    try:
         
        res['A_close to B-1_close'] = round(((final_df.iloc[i]['X-1(close)'] - final_df.iloc[i-1]['X(close)'])/final_df.iloc[i-1]['X(close)'])*100, 3)
        res['B-1_close to B_close'] = round(((final_df.iloc[i]['X(close)'] - final_df.iloc[i]['X-1(close)'])/final_df.iloc[i]['X-1(close)'])*100, 3)
        res['A+1_open to B-1_close'] = round(((final_df.iloc[i]['X-1(close)'] - final_df.iloc[i-1]['X+1(open)'])/final_df.iloc[i-1]['X+1(open)'])*100, 3)
        
    except:
        pass
    i += 1
    report_df = report_df.append(res, ignore_index=True)
   
    
report_df[symbol] = final_df[symbol]

expiry_dates = pd.read_excel(master_dir+'Expiry_dates_master.xlsx')
expiry_dates = expiry_dates[expiry_dates['Expiry']=='E']
expiry_dates['Dates'] = expiry_dates.apply(lambda row: parser.parse(str(row['Date'])+' '+row['Month']+' '+str(row['Year'])).date(), axis=1)
# exclude expiry dates
report_df = report_df[~report_df['BANKNIFTY'].isin(expiry_dates['Dates'].values)]


report_df.set_index(symbol, inplace=True)
report_df.fillna(0, inplace=True)
# remove expiry dates from the report 
expiry_dates = pd.read_excel(master_dir+'Expiry_dates_master.xlsx')

report_df.to_csv('report.csv')



'''

result = pd.DataFrame()
for anchor_date in anchor_dates['Dates']:
    if len(df[df['price_date']==anchor_date])!=0:
        result = result.append(df[df['price_date']==anchor_date])
        

result = result[['price_date','open','close']]    

'''

"\n\nresult = pd.DataFrame()\nfor anchor_date in anchor_dates['Dates']:\n    if len(df[df['price_date']==anchor_date])!=0:\n        result = result.append(df[df['price_date']==anchor_date])\n        \n\nresult = result[['price_date','open','close']]    \n\n"

In [166]:
final_df.head(2)

BANKNIFTY  X(close) X+1(close) X-1(close) X+1(open)
0  2018-05-03  25711.75   25701.15          0  25679.25
1  2018-05-10   26072.9   26400.35   26128.45   26115.0

In [167]:
report_df.head(2)

A_close to B-1_close  B-1_close to B_close  A+1_open to B-1_close
BANKNIFTY                                                                    
2018-05-03              -100.000                 0.000                  0.000
2018-05-10                 1.621                -0.213                  1.749

In [18]:
dates = df['price_date']
result = pd.DataFrame()
# before anchor date
anchor_value = df[df['price_date'] == anchor_date][value].values[0]
    
for index in list(reversed(days_index)):
    try:
        result['X-{0}({1})'.format(index, value)] = df[df['price_date'] == dates[dates[dates == anchor_date].index[0] - index].date()][value].values[0] 
    except:
        logging.info('Not in index {0}'.format(index))
    

result['X({0})'.format(value)] = anchor_value

# after anchor date
for index in days_index:
    try:
        result['X+{0}({1})'.format(index, value)] = df[df['price_date'] == dates[dates[dates == anchor_date].index[0] + index].date()][value].values[0] 
    except:
        logging.info('Not in index {0}'.format(index))

res = pd.DataFrame(result, index=[anchor_date], columns=result.keys())


    

In [37]:
df[df['price_date'] == anchor_date][values].values[0][0]

Decimal('26310.1')

In [29]:
values = [ 'open', 'close']

In [24]:
values

['open', 'close']

In [5]:
import pandas as pd
#import numpy as np
#import os
from dateutil import parser
from collections import OrderedDict
from cassandra.cluster import Cluster
import logging 
from time import time
import datetime


input_dir = 'X:\\Data_Analytics\\BankNifty\\input\\'
output_dir = 'X:\\Data_Analytics\\BankNifty\\Output\\'
master_dir = 'X:\\Data_Analytics\\BankNifty\\Master\\'
log_path = "X:\\Data_Analytics\\BankNifty\\"

# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")



def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)


def get_values(min_date, max_date, key):
    '''Function to retrive values from cassandra db'''
    
    cluster = Cluster(['127.0.0.1'])
    logging.info('Cassandra Cluster connected...')
    # connect to your keyspace and create a session using which u can execute cql commands 
    session = cluster.connect('test_df')
    logging.info('Using test_df keyspace')
    #define rows fetched from cassandra to be a pandas dataframe
    session.row_factory = pandas_factory
    session.default_fetch_size = None    

    query = 'SELECT * FROM test_bhavcopy WHERE price_date >= \'{0}\' AND price_date <= \'{1}\' AND key = \'{2}\'ALLOW FILTERING;'.format(
                        min_date,max_date,key)

    rows = session.execute(query, timeout = None)
    rows = rows._current_rows
    rows.drop(columns=['key','option_typ','strike_pr'], inplace=True)
    return rows
    
def anchor_buckets_func(value, anchor_date, days_index, symbol, df):
    '''Function to get bucket values'''
    
    dates = df['price_date']
    result = OrderedDict()
    # before anchor date
    anchor_value = df[df['price_date'] == anchor_date][value].values[0]
    
    for index in list(reversed(days_index)):
        try:
            result['X-{0}({1})'.format(index, value)] = df[df['price_date'] == dates[dates[dates == anchor_date].index[0] - index].date()][value].values[0] 
        except:
            logging.info('Not in index {0}'.format(index))
    

    result['X({0})'.format(value)] = anchor_value

    # after anchor date
    for index in days_index:
        try:
            result['X+{0}({1})'.format(index, value)] = df[df['price_date'] == dates[dates[dates == anchor_date].index[0] + index].date()][value].values[0] 
        except:
            logging.info('Not in index {0}'.format(index))

    res = pd.DataFrame(result, index=[anchor_date], columns=result.keys())
    
    return res


def min_max_calc(min_date, max_date, anchor_date):
    '''Func to handle absolute days for min and max dates'''
    min_date = min_date
    max_date = max_date
    if len(min_date)<=3:
        # handle abs days
        min_date = anchor_date - datetime.timedelta(days=int(min_date))
    else:
        try:
            min_date = parser.parse(min_date).date()
        except:
            logging.info('Out of range for current month')
        
    if len(max_date)<=3:
        #handle abs days
        max_date = anchor_date + datetime.timedelta(days=int(max_date))
    else:
        try:
            max_date = parser.parse(max_date).date()
        except:
            logging.info('Out of range for current month')
        
    return min_date, max_date
        
        



def main():
    
    # Get inputs
    symbol = str.upper('BANKNIFTY')
    anchor_dates = pd.read_excel(input_dir+'input.xlsx')    
    min_date, max_date = min(anchor_dates['Dates']).date() , max(anchor_dates['Dates']).date()
    
    anchor_dates['Dates'] = anchor_dates.apply(lambda row: row['Dates'].date(), axis=1)
    #min_date = raw_input('min date')
    #max_date = raw_input('max date')
    #min_date,max_date = min_max_calc(min_date, max_date, anchor_date)
    
    # fetch keys from master 
    keys = pd.read_excel(master_dir+'MasterData.xlsx')
    keys = keys[keys['IsActiveFNO']==True]
    keys.loc[keys['Type']=='SSF','instrument'] = 'FUTSTK'
    keys.loc[keys['Type']=='Index','instrument'] = 'FUTIDX'
    keys = keys['SYMBOL']+"_"+keys['instrument']+"_0.0_XX_1"
    
    # fetch key for symbol
    key = keys[keys.str.startswith(symbol+'_')].values[0]
    
    
    # get values from cassandra
    df = get_values(min_date,max_date,key)    
    days_index = [1]
    # get anchor+-1 close prices 
    close_df = pd.DataFrame()
    for anchor_date in anchor_dates['Dates']:
        close_df = close_df.append(anchor_buckets_func('close', anchor_date, days_index, symbol, df))
        
    close_df.index.name = symbol    
    # get anchor+-1 close prices
    open_df = pd.DataFrame()
    for anchor_date in anchor_dates['Dates']:
        open_df = open_df.append(anchor_buckets_func('open', anchor_date, days_index, symbol, df))
        
    open_df.index.name = symbol
    # get only X+1 open values 
    open_df = open_df[['X+1(open)']]
    # merge both the df
    final_df = close_df.merge(open_df, on=symbol)
    final_df.fillna(0, inplace=True)
    final_df.reset_index(inplace=True)
    
    
    # get report 
    report_df = pd.DataFrame(columns=['A_close to B-1_close','B-1_close to B_close', 'A+1_open to B-1_close' ])
    anchor_dates = final_df.index
    
    i = 0
    for anchor_date in final_df[symbol]:
        res = {}
        try:
             
            res['A_close to B-1_close'] = round(((final_df.iloc[i]['X-1(close)'] - final_df.iloc[i-1]['X(close)'])/final_df.iloc[i-1]['X(close)'])*100, 3)
            res['B-1_close to B_close'] = round(((final_df.iloc[i]['X(close)'] - final_df.iloc[i]['X-1(close)'])/final_df.iloc[i]['X-1(close)'])*100, 3)
            res['A+1_open to B-1_close'] = round(((final_df.iloc[i]['X-1(close)'] - final_df.iloc[i-1]['X+1(open)'])/final_df.iloc[i-1]['X+1(open)'])*100, 3)
            
        except:
            pass
        i += 1
        report_df = report_df.append(res, ignore_index=True)
       
        
    report_df[symbol] = final_df[symbol]
    
    expiry_dates = pd.read_excel(master_dir+'Expiry_dates_master.xlsx')
    expiry_dates = expiry_dates[expiry_dates['Expiry']=='E']
    expiry_dates['Dates'] = expiry_dates.apply(lambda row: parser.parse(str(row['Date'])+' '+row['Month']+' '+str(row['Year'])).date(), axis=1)
    # exclude expiry dates
    report_df = report_df[~report_df['BANKNIFTY'].isin(expiry_dates['Dates'].values)]
    
    
    report_df.set_index(symbol, inplace=True)
    report_df.fillna(0, inplace=True)
    # remove expiry dates from the report 
    expiry_dates = pd.read_excel(master_dir+'Expiry_dates_master.xlsx')
    
    report_df.to_csv(output_dir+'{0}_report.csv'.format(symbol))
    
    
    
    
start_time = time()

if __name__ == '__main__':
    main()

end_time = time()

logging.info('Time taken to process :'.format(end_time - start_time))
print "Execution time: {0} Seconds.... ".format(end_time - start_time)


          

    


Execution time: 1.30500006676 Seconds.... 
